In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# import data
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
# make dataframe
df_raw = pd.read_csv(url)

#only keep specific columns
df = df_raw[['engine_displacement',
'horsepower',
'vehicle_weight',
'model_year',
'fuel_efficiency_mpg']].copy()

#preview
df.head()

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,170,159.0,3413.433759,2003,13.231729
1,130,97.0,3149.664934,2007,13.688217
2,170,78.0,3079.038997,2018,14.246341
3,220,NaN,2542.392402,2009,16.912736
4,210,140.0,3460.870990,2009,12.488369


In [ ]:
## Previewing and formatting ##

#preview columns for null values
df.isnull().sum()
# median of horsepower 149
df['horsepower'].median()

# add horsepower adjusted fields
df['horsepower_adj_zero'] = df['horsepower'].fillna(0)
#do mean later, calculated using training dataset only
#df['horsepower_adj_mean'] = df['horsepower'].fillna(df['horsepower'].mean())

np.float64(149.0)

In [ ]:
## Splitting data and subbing na for mean using train ##

# splitting the dataset into training, validate and test sets
n = len(df)
n_val = int(len(df) * 0.2)
n_test = int(len(df) * 0.2)
n_train = n - n_val - n_test

# make sure they are split to equal total recores
n_val, n_test, n_train, n_val + n_test + n_train, len(df)

#shuffle records by setting a randon sequence
idx = np.arange(n)
np.random.seed(42) # makes the random reproducable
np.random.shuffle(idx)

#select random rows based on random sequence
df_train = df.iloc[idx[:n_train]]
df_val = df.iloc[idx[n_train:n_train + n_val]]
df_test = df.iloc[idx[n_train + n_val:]]

#mean using training dataset only
df['horsepower_adj_mean'] = df_train['horsepower'].fillna(df['horsepower'].mean())

#test split
len(df_train), len(df_val), len(df_test), len(df_train) + len(df_val) + len(df_test), len(df)

(5824, 1940, 1940, 9704, 9704)